# Imports

In [ ]:
%matplotlib qt
%load_ext autoreload   
%autoreload 2


In [ ]:
from vpm_py import VPM
import numpy as np
from mpi4py import MPI
import numpy as np

from vpm_py.console_io import print_IMPORTANT, print_red, print_blue, print_green
from vpm_py.visualization import StandardVisualizer
from test_hill_problem import hill_assign_parallel, visualize_vorticity


# Initialize VPM

In [ ]:
# Initialize MPI
comm = MPI.COMM_WORLD
start_time = MPI.Wtime()
rank = comm.Get_rank()
np_procs = comm.Get_size()

# Initialize VPM
vpm = VPM(
    number_of_equations=3,
    number_of_processors=np_procs,
    rank=rank,
    verbocity=0,
    dx_particle_mesh=0.1,
    dy_particle_mesh=0.1,
    dz_particle_mesh=0.1,
)
plotter = StandardVisualizer(
    plot_particles=("charge", "magnitude"),  # plot_mesh=("velocity", "magnitude")
)

# PRINT THE RANK OF THE PROCESS AND DETERMINE HOW MANY PROCESSES ARE RUNNING
print_blue(f"Number of processes: {np_procs}", rank)
comm.Barrier()
print_blue(f"Rank: {rank}")
comm.Barrier()


In [ ]:
DT = 1 * 0.1
NI = -0.1
neq = 3
UINF = np.array([0.0, 0.0, 0.0])

# Create particles
NVR = 100
XPR_zero = np.zeros((3, NVR), dtype=np.float64)
XPR_zero[:, 0] = np.array([-2, -2, -20])
XPR_zero[:, 1] = np.array([2, 2, 20])
QPR_zero = np.ones((neq + 1, NVR), dtype=np.float64)
UPR_zero = np.zeros((3, NVR), dtype=np.float64)
GPR_zero = np.zeros((3, NVR), dtype=np.float64)


In [ ]:
# Initialization VPM
comm.Barrier()
vpm.vpm(
    num_equations=neq,
    mode=0,
    particle_positions=XPR_zero,
    particle_charges=QPR_zero,
    timestep=0,
    viscosity=NI,
)
comm.Barrier()


# Initialize Hill Vortex

In [ ]:
if rank == 0:
    st = MPI.Wtime()

print_IMPORTANT(f"Hill vortex initialization", rank)
_, RHS_pm_1 = hill_assign_parallel(
    Dpm=vpm.dpm,
    NN=vpm.particle_mesh.nn,
    NN_bl=vpm.particle_mesh.nn_bl,
    Xbound=vpm.particle_mesh.xbound,
    neqpm=vpm.num_equations,
    sphere_radius=1.5,
    u_freestream=1.0,
    sphere_z_center=0.0,
)
RHS_pm_hill = RHS_pm_1
# visualize_vorticity(RHS_pm_hill, vpm.nn_bl)


In [ ]:
vpm.particle_mesh.set_rhs_pm(RHS_pm_hill)
print_red(f"Setting RHS_PM as computed from the hill vortex", rank)

if rank == 0:
    st = MPI.Wtime()
    print_red(f"Remeshing")
XPR_zero, QPR_zero = vpm.remesh_particles(project_particles=False)
if rank == 0:
    et = MPI.Wtime()
    print(f"\tRemeshing finished in {int((et - st) / 60)}m {int(et - st) % 60}s\n")

print_IMPORTANT(f"Particles initialized", rank)

# Create the plot to live update the particles
if rank == 0:
    plotter.update_particle_plots(
        iteration=0,
        particle_positions=XPR_zero[:, :],
        particle_velocities=UPR_zero[:, :],
        particle_charges=QPR_zero[:, :],
        particle_deformations=GPR_zero[:, :],
    )


# Simulate

In [ ]:
vpm.vpm(
    num_equations= vpm.num_equations,
    mode = 0,
    particle_positions  =  XPR_zero[:,:],
    particle_charges    =  QPR_zero[:,:],
    timestep=0,
    viscosity=NI,
)
# Main loop
T = 0
max_iter = 500
XPR = XPR_zero.copy()
QPR = QPR_zero.copy()
i = 0


In [ ]:
i += 1
NVR = vpm.particles.NVR
XPR = vpm.particles.XP
QPR = vpm.particles.QP

T += DT
print_IMPORTANT(
    f"Iteration= {i} of {max_iter}\nT={T}\nDT={DT}",
    rank = rank,
    color_divider="green",
    color_text="green"
)
vpm.vpm(
    num_equations=neq,
    mode = 2,
    particle_positions    =  XPR,
    particle_charges      =  QPR,
    timestep=i,
    viscosity=NI,
)


In [ ]:
print_IMPORTANT(f"INFO", rank)
XPR = vpm.particles.XP
QPR = vpm.particles.QP
UPR = vpm.particles.UP.to_numpy(copy=True)
GPR = vpm.particles.GP.to_numpy(copy=True)
# Print the size of the particles
# print(f"Number of particles: {NVR}")
# print(f"Number of equations: {neq}")
# print('\n')

# print_green(f"UPR:")
# print(f"Mean: {np.mean(UPR.data, axis=1)}")
# print(f"Max: {np.max(UPR.data, axis=1)}")
# print(f"Min: {np.min(UPR.data, axis=1)}")
# print('\n')

U_PM = vpm.particle_mesh.U
for name, u in zip(["Ux", "Uy", "Uz"], U_PM): 
    print_green(f"{name}:")
    print(f"Mean: {np.mean(u)}")
    print(f"Max: {np.max(u)}")
    print(f"Min: {np.min(u)}")
    print('\n')

print_IMPORTANT(f"Convecting Particles", rank)

st = MPI.Wtime()
# # Move the particles
for j in range(vpm.particles.NVR):
    # Translate the particles
    XPR[:3, j] = XPR[:3, j] + UPR[:3,j] * DT
    pass
    # QPR[:3, j] -= GPR[:3, j] * DT
et = MPI.Wtime()

print(f"\tConvection finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")
print_IMPORTANT(f"Updating the plot", rank)

st = MPI.Wtime()
# Update the plot
plotter.update_particle_plots(
    iteration=i,
    particle_positions= XPR[:,:],
    particle_charges= QPR[:,:],
    particle_velocities= UPR[:,:],
    particle_deformations= GPR[:,:]
)
plotter.update_mesh_plots(
    iteration=i,
    pm_positions= vpm.particle_mesh.grid_positions,
    pm_velocities= vpm.particle_mesh.U,
    pm_charges= vpm.particle_mesh.RHS,
    pm_deformations= vpm.particle_mesh.deformation
)
et = MPI.Wtime()
print(f"\tUpdating the plot finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")
print_IMPORTANT(f"Saving the particles and particle mesh", rank)

st = MPI.Wtime()
vpm.particles.save_to_file(filename= f"particles_test", folder="vpm_case")
vpm.particle_mesh.save_to_file(filename= f"particle_mesh_test", folder="vpm_case")
et = MPI.Wtime()
print(f"\tSaving the particles and particle mesh finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")


## Remesh and Redefine

In [ ]:
vpm.vpm(
    num_equations=neq,
    mode = 0,
    particle_positions  =  XPR,
    particle_charges    =  QPR,
    timestep=i,
    viscosity=NI,
)
# XPR, QPR = vpm.remesh_particles(project_particles=True, cut_off=1e-5)


In [ ]:
for i in range(3, 100):
    NVR = vpm.particles.NVR
    XPR = vpm.particles.XP
    QPR = vpm.particles.QP

    T += DT
    print_IMPORTANT(
        f"Iteration= {i} of {max_iter}\nT={T}\nDT={DT}",
        rank = rank,
        color_divider="green",
        color_text="green"
    )
    vpm.vpm(
        num_equations=neq,
        mode = 2,
        particle_positions    =  XPR,
        particle_charges      =  QPR,
        timestep=i,
        viscosity=NI,
    )

    print_IMPORTANT(f"INFO", rank)
    XPR = vpm.particles.XP
    QPR = vpm.particles.QP
    UPR = vpm.particles.UP.to_numpy(copy=True)
    GPR = vpm.particles.GP.to_numpy(copy=True)
    # Print the size of the particles
    # print(f"Number of particles: {NVR}")
    # print(f"Number of equations: {neq}")
    # print('\n')

    # print_green(f"UPR:")
    # print(f"Mean: {np.mean(UPR.data, axis=1)}")
    # print(f"Max: {np.max(UPR.data, axis=1)}")
    # print(f"Min: {np.min(UPR.data, axis=1)}")
    # print('\n')

    U_PM = vpm.particle_mesh.U
    for name, u in zip(["Ux", "Uy", "Uz"], U_PM): 
        print_green(f"{name}:")
        print(f"Mean: {np.mean(u)}")
        print(f"Max: {np.max(u)}")
        print(f"Min: {np.min(u)}")
        print('\n')

    print_IMPORTANT(f"Convecting Particles", rank)

    st = MPI.Wtime()
    # # Move the particles
    for j in range(vpm.particles.NVR):
        # Translate the particles
        XPR[:3, j] = XPR[:3, j] + UPR[:3,j] * DT
        pass
        # QPR[:3, j] -= GPR[:3, j] * DT
    et = MPI.Wtime()

    print(f"\tConvection finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")
    print_IMPORTANT(f"Updating the plot", rank)

    st = MPI.Wtime()
    # Update the plot
    plotter.update_particle_plots(
        iteration=i,
        particle_positions= XPR[:,:],
        particle_charges= QPR[:,:],
        particle_velocities= UPR[:,:],
        particle_deformations= GPR[:,:]
    )
    plotter.update_mesh_plots(
        iteration=i,
        pm_positions= vpm.particle_mesh.grid_positions,
        pm_velocities= vpm.particle_mesh.U,
        pm_charges= vpm.particle_mesh.RHS,
        pm_deformations= vpm.particle_mesh.deformation
    )
    et = MPI.Wtime()
    print(f"\tUpdating the plot finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")
    print_IMPORTANT(f"Saving the particles and particle mesh", rank)

    st = MPI.Wtime()
    vpm.particles.save_to_file(filename= f"particles_test", folder="vpm_case")
    vpm.particle_mesh.save_to_file(filename= f"particle_mesh_test", folder="vpm_case")
    et = MPI.Wtime()
    print(f"\tSaving the particles and particle mesh finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")
    vpm.vpm(
        num_equations=neq,
        mode = 0,
        particle_positions  =  XPR,
        particle_charges    =  QPR,
        timestep=i,
        viscosity=NI,
    )
